In [ ]:
#========================================================================
# MLM(Masked Language Model) 과 NSP(Next Sentence Predict) 로 Further Pre-Train 시키는 예시
# 참고예제 : https://towardsdatascience.com/how-to-train-bert-aaad00533168
#========================================================================

import os
from os import sys
from tqdm.notebook import tqdm
from transformers import BertTokenizer, BertForPreTraining, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, RandomSampler

sys.path.append('..')
from myutils import GPU_info, seed_everything, mlogging, AccuracyForMaskedToken, SaveBERTModel

device = GPU_info()
print(device)

#seed 설정
seed_everything(111)

#logging 설정
logger =  mlogging(loggername="bertfptnspmlm", logfilename="../../log/bertfptnspmlm")

# 훈련시킬 말뭉치(사전 만들때 동일한 말뭉치 이용)
# 훈련시킬 말뭉치는  .으로 구분된 한줄 문자이 아니라. 한줄에 .로구분된 여러문장이 이어진 문장이어야 함
# 예시:'제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.
input_corpus = "../../data/my_data/data.txt"

# eval 말뭉치 
eval_corpus = "../../korpora/kowiki_20190620/wiki_eval_test.txt"

# 기존 사전훈련된 모델
model_path = "../../model/bert/bert-multilingual-cased"

# 기존 사전 + 추가된 사전 파일
#vocab_path="tokenizer/wiki_20190620_false_0311_speical/bmc_add_wiki_20190620_false_0311.txt"
vocab_path="../../tokenizer/my_bong_vocab/"

# 출력
OUTPATH = '../../model/bert/bert-multilingual-cased-0418/'

batch_size = 32
token_max_len = 128


In [ ]:
# tokenizer와 모델 로딩
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained(vocab_path, max_len=token_max_len, do_lower_case=False)
print('special_token_size: {}, tokenizer.vocab_size: {}'.format(len(tokenizer.all_special_tokens), tokenizer.vocab_size))
print('tokenizer_len: {}'.format(len(tokenizer)))

#model = BertForPreTraining.from_pretrained('bert-base-multilingual-cased')
model = BertForPreTraining.from_pretrained(model_path)    

model.resize_token_embeddings(len(tokenizer))
model.to(device)

In [ ]:
# test Data 불러옴.
# test data는 .으로 구분된 한줄 문자이 아니라. 한줄에 .로구분된 여러문장이 이어진 문장이어야 함
# 예시:'제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.
with open(input_corpus, 'r') as fp:
    text = fp.read().split('\n')
    
print(text[:2])

In [ ]:
# NSP를 만들기 위해, .을 기준으로 문장들을 나눈 후 길이를 얻어 둔다.
bag = [item for sentence in text for item in sentence.split('.') if item != '']
bag_size = len(bag)
print(bag_size)
print(bag[14])

In [ ]:
#NSP는 50:50으로 랜덤한 값, 랜덤하지 않은 문장으로 만든다.
import random

sentence_a = []
sentence_b = []
label = []

for paragraph in text:
    # 하나의 문장을 읽어와서 .기준으로 나눈다.
    sentences = [sentence for sentence in paragraph.split('.') if sentence != '']
    num_sentences = len(sentences)
    
     # . 기준으로 나눈 문장이 1이상이면..
    if num_sentences > 1:
        # 문장 a 시작번지는 랜덤하게, 해당 문장 이후로 지정
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        # 0.5 이상 랜덤값이면, 연속적인 문장으로 만듬
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            label.append(0)  #label=0이면 연속적
        # 0.5 이하 랜덤값이면  연속적이 아닌 문장으로 만듬
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index])
            label.append(1)  #label=1이면 비연속적

In [ ]:
# label = 0 이면, 연속적인 문장, 1이면 연속적인 문장이 아님
for i in range(3):
    print(label[i])
    print(sentence_a[i] + '\n---')
    print(sentence_b[i] + '\n')

In [ ]:
# 위 NSP 리스트 들을 tokenizer 함

# max_length = 512 하면 GPU Memory 오류 발생함
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt',
                   max_length=token_max_len, truncation=True, padding='max_length')

print(inputs.keys())

print(inputs)

In [ ]:
# tokenizer 한 NSP 에 'next_sentence_label' 값(0,1) 추가함
inputs['next_sentence_label'] = torch.LongTensor([label]).T

print(inputs.next_sentence_label[:10])

In [ ]:
# MLM 만들기

# labels에는 inputs_id를 복사해서 추가
inputs['labels'] = inputs.input_ids.detach().clone()

print(inputs.keys())

In [ ]:
# 각 스페셜 tokenid를 구함
CLStokenid = tokenizer.convert_tokens_to_ids('[CLS]')
SEPtokenid = tokenizer.convert_tokens_to_ids('[SEP]')
UNKtokenid = tokenizer.convert_tokens_to_ids('[UNK]')
PADtokenid = tokenizer.convert_tokens_to_ids('[PAD]')
MASKtokenid = tokenizer.convert_tokens_to_ids('[MASK]')
print('CLSid:{}, SEPid:{}, UNKid:{}, PADid:{}, MASKid:{}'.format(CLStokenid, SEPtokenid, UNKtokenid, PADtokenid, MASKtokenid))

# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != CLStokenid) * (inputs.input_ids != SEPtokenid) * \
           (inputs.input_ids != UNKtokenid) * (inputs.input_ids != PADtokenid) * \
           (inputs.input_ids != MASKtokenid)

selection = []
for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )
    
print(selection[:2])

# inputs_ids 에 [MASK] 추가시킴
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = MASKtokenid
    

print(inputs.keys())
print(inputs.input_ids)


In [ ]:
# 훈련 dataloader 만듬 
#batch_size = 16

class OurDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)
    
train_dataset = OurDataset(inputs)

train_loader = DataLoader(train_dataset, 
                          batch_size=batch_size, 
                          #shuffle=True,
                          sampler=RandomSampler(train_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                          num_workers=3)

In [ ]:
import sys
sys.path.append('..')
from myutils import MLMDataset

#===============================================================================
# eval dataloader 생성
eval_dataset = MLMDataset(corpus_path = eval_corpus,
                          tokenizer = tokenizer, 
                          CLStokeinid = CLStokenid ,   # [CLS] 토큰 id
                          SEPtokenid = SEPtokenid ,    # [SEP] 토큰 id
                          UNKtokenid = UNKtokenid ,    # [UNK] 토큰 id
                          PADtokenid = PADtokenid,    # [PAD] 토큰 id
                          Masktokenid = MASKtokenid,   # [MASK] 토큰 id
                          max_sequence_len=token_max_len,  # max_sequence_len)
                          mlm_probability=0.15,
                          overwrite_cache=False
                          )


# eval dataloader 생성
# => tenosor로 만듬
eval_loader = DataLoader(eval_dataset, 
                         batch_size=batch_size, 
                         #shuffle=True, # dataset을 섞음
                         sampler=RandomSampler(eval_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                         num_workers=3
                         )
#===============================================================================

print(eval_dataset[0])

In [ ]:
# 훈련 시작
##################################################
epochs = 5
learning_rate = 3e-5  # 학습률
##################################################

# optimizer 적용
optimizer = AdamW(model.parameters(), 
                 lr=learning_rate, 
                 eps=1e-8) # 0으로 나누는 것을 방지하기 위한 epsilon 값(10^-6 ~ 10^-8 사이 이값 입력합)

# 총 훈련과정에서 반복할 스탭
total_steps = len(train_loader)*epochs
warmup_steps = int(total_steps * 0.1) #10% of train data for warm-up

# 손실률 보여줄 step 수
p_itr = int(len(train_loader)*0.1)  
if p_itr <= 0:
    p_itr = 1
    
# step마다 모델 저장
save_steps = int(total_steps * 0.5)

logger.info('*epchos:{}, lr:{:.9f}, total_steps: {}, warmup_steps:{}, p_itr:{}, save_steps:{}'.format(epochs, learning_rate, total_steps, warmup_steps, p_itr, save_steps))
               
# 스캐줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=warmup_steps, 
                                            num_training_steps=total_steps)

itr = 1
total_loss = 0
total_len = 0
total_correct = 0
total_test_correct = 0
total_test_len = 0
    
list_train_loss = []
list_train_acc = []
list_validation_acc = []

model.zero_grad()# 그래디언트 초기화
for epoch in tqdm(range(epochs)):
    # setup loop with TQDM and dataloader
    #loop = tqdm(train_loader, leave=True)
    
    #for batch in loop:
    model.train() # 훈련모드로 변환
    for data in tqdm(train_loader):
        # initialize calculated gradients (from prev step)
        model.zero_grad()# 그래디언트 초기화
    
        # pull all tensor batches required for training
        input_ids = data['input_ids'].to(device)
        token_type_ids = data['token_type_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        next_sentence_label = data['next_sentence_label'].to(device)
        labels = data['labels'].to(device)
        
        # process
        outputs = model(input_ids = input_ids, 
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        logits = outputs.prediction_logits # torch.Size([32, 128, 157660]) => [batch_size, sequence_length, vocab_size]
        #print(logits.shape)
        
        # calculate loss for every parameter that needs grad update
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   # 그래디언트 클리핑 (gradient vanishing이나 gradient exploding 방지하기 위한 기법)      
        # update parameters
        optimizer.step()
        scheduler.step()  # 학습률 감소
        
        
        # print relevant info to progress bar
        #loop.set_description(f'Epoch {epoch}')
        #loop.set_postfix(loss=loss.item())
        
        # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
        # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
        with torch.no_grad():
            
            # 손실률 계산
            total_loss += loss.item()
            
            #===========================================
            # 정확도(Accurarcy) 계산
            correct, masked_len = AccuracyForMaskedToken(logits, labels, input_ids, MASKtokenid)           
            total_correct += correct.sum().item() 
            total_len += masked_len 
            #=========================================
                
            # 주기마다 test(validataion) 데이터로 평가하여 손실류 계산함.
            if itr % p_itr == 0:
                
                train_loss = total_loss/p_itr
                train_acc = total_correct/total_len
                       
                ####################################################################
                # 주기마다 eval(validataion) 데이터로 평가하여 손실류 계산함.
                # 평가 시작
                model.eval()

                #for data in tqdm(eval_loader):
                for data in eval_loader:
                    # 입력 값 설정
                    input_ids = data['input_ids'].to(device)
                    attention_mask = data['attention_mask'].to(device)
                    token_type_ids = data['token_type_ids'].to(device)       
                    labels = data['labels'].to(device)

                    with torch.no_grad():
                        # 모델 실행
                        outputs = model(input_ids=input_ids, 
                                       attention_mask=attention_mask,
                                       token_type_ids=token_type_ids,
                                       labels=labels)

                        # 출력값 loss,logits를 outputs에서 얻어옴
                        #loss = outputs.loss
                        logits = outputs.prediction_logits 

                        #===========================================
                        # 정확도(Accurarcy) 계산
                        correct, masked_len = AccuracyForMaskedToken(logits, labels, input_ids, MASKtokenid)           
                        total_test_correct += correct.sum().item() 
                        total_test_len += masked_len 
                        #=========================================

                val_acc = total_test_correct/total_test_len
                    
                logger.info('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Train Acc: {:.4f}, Val Acc:{}({}/{})'.format(epoch+1, epochs, itr, train_loss, train_acc, val_acc, total_test_correct, total_test_len))
      
                list_train_loss.append(train_loss)
                list_train_acc.append(train_acc)
                list_validation_acc.append(val_acc)
                 
                # 변수들 초기화    
                total_loss = 0
                total_len = 0
                total_correct = 0
                total_test_correct = 0
                total_test_len = 0
                ####################################################################

            if itr % save_steps == 0:
                #전체모델 저장
                SaveBERTModel(model, tokenizer, OUTPATH, epochs, learning_rate, batch_size)
                
        itr+=1

In [ ]:
# 그래프로 loss 표기
#!pip install matplotlib
import matplotlib.pyplot as plt

plt.plot(list_train_loss, label='Train Loss')
#plt.plot(list_train_acc, label='Train Accuracy')
#plt.plot(list_validation_acc, label='Eval Accuracy')
plt.legend()
plt.show()

plt.plot(list_train_acc, label='Train Accuracy')
plt.plot(list_validation_acc, label='Eval Accuracy')
plt.legend()
plt.show()

In [ ]:
# 모델 저장
#save_model(model, tokenizer, OUTPATH, epochs, learning_rate, batch_size)